# Imports

In [74]:
import numpy as np # linear algebra
np.random.seed(666)
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from subprocess import check_output

# utils

In [75]:
def evaluate_model(model, x, y):
    res = model.evaluate(x,y)
    print('loss:',res[0], 'acc:', res[1])
    
def write_subs(x, ids, filename = 'subs'):
    test_predictions = model.predict(x)
    test_predictions = test_predictions.reshape((test_predictions.shape[0]))
    test_predictions = ['{:f}'.format(item) for item in test_predictions]

    submission = pd.DataFrame({'id': ids, 'is_iceberg': test_predictions})
    submission.to_csv(filename, index = False)
    return submission

# data

In [116]:
#Load data
train = pd.read_json("train.json")
test = pd.read_json("test.json")
train.inc_angle = train.inc_angle.replace('na', 0)
train.inc_angle = train.inc_angle.astype(float).fillna(0.0)
print("done!")

ValueError: Could not reserve memory block

In [118]:
# Train data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_train = np.concatenate([x_band1[:, :, :, np.newaxis]
                          , x_band2[:, :, :, np.newaxis]], axis=-1)
X_angle_train = np.array(train.inc_angle)
y_train = np.array(train["is_iceberg"])

# Test data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_test = np.concatenate([x_band1[:, :, :, np.newaxis]
                          , x_band2[:, :, :, np.newaxis]], axis=-1)
X_angle_test = np.array(test.inc_angle)


X_train, X_valid, X_angle_train, X_angle_valid, y_train, y_valid = train_test_split(X_train
                    , X_angle_train, y_train, random_state=123, train_size=0.6)
X_valid, X_testing, y_valid, y_testing = train_test_split(X_valid, y_valid, random_state=123, train_size=0.5)

In [109]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPool2D
from keras.layers.normalization import BatchNormalization 
from keras.utils import np_utils
from keras.datasets import mnist
import keras

In [7]:
acti = 'relu'

In [8]:
model = Sequential()

model.add(BatchNormalization(input_shape=(75, 75, 3)))

model.add(Conv2D(32, kernel_size=(3,3), activation=acti))
model.add(Conv2D(32, kernel_size=(3,3), activation=acti))
model.add(Conv2D(32, kernel_size=(3,3), activation=acti))
model.add(Conv2D(32, kernel_size=(3,3), activation=acti))
model.add(MaxPooling2D(pool_size=(2,2)))
#model.add(Dropout(0.1))
model.add(BatchNormalization())

model.add(Conv2D(48, kernel_size=(3,3), activation=acti))
model.add(Conv2D(48, kernel_size=(3,3), activation=acti))
model.add(Conv2D(48, kernel_size=(3,3), activation=acti))
model.add(MaxPooling2D(pool_size=(2,2)))
#model.add(Dropout(0.1))
model.add(BatchNormalization())


model.add(Conv2D(64, kernel_size=(3,3), activation=acti))
model.add(Conv2D(64, kernel_size=(3,3), activation=acti))
model.add(Conv2D(128, kernel_size=(3,3), activation=acti))
model.add(GlobalMaxPool2D())
#model.add(Dropout(0.1))
model.add(BatchNormalization())


model.add(Dense(300, activation=acti))
model.add(Dropout(0.4)) 
model.add(BatchNormalization())


model.add(Dense(128, activation=acti))
model.add(Dropout(0.4))
model.add(BatchNormalization())


model.add(Dense(1, activation='sigmoid'))

In [79]:
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

def get_callbacks(filepath, patience=10):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    
    return [es, msave]

callbacks = get_callbacks('./model.hdf5')

# Using Datagen

In [13]:
from keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(
        rotation_range=70,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        rescale=1./255,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest')


test_gen = ImageDataGenerator(rescale=1./255)


training_set = train_gen.flow(X_train, y_train, batch_size=32)
valid_set = test_gen.flow(X_valid, y_valid, batch_size=32)
test_set = test_gen.flow(X_test, batch_size=1, shuffle=False)

In [13]:
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.adam(lr=0.001, decay=0.25), metrics=['mae'])
model.fit_generator(training_set, steps_per_epoch=3000, epochs=1000, 
                    callbacks=callbacks, validation_data=valid_set, 
                    validation_steps=400, workers=7)

Epoch 1/1000
 327/3000 [==>...........................] - ETA: 252s - loss: 0.6811 - mean_absolute_error: 0.4050- E

KeyboardInterrupt: 

In [545]:
print(model.evaluate_generator(training_set, 4000))
print(model.evaluate_generator(valid_set, 400))

[0.25241673039867457, 0.16393957372945869]
[0.27912027280087598, 0.17202319949321723]


In [550]:
test_predictions = model.predict_generator(test_set, steps = test.shape[0], use_multiprocessing=True)
test_predictions = test_predictions.reshape((test_predictions.shape[0]))
test_predictions = ['{:f}'.format(item) for item in test_predictions]

submission = pd.DataFrame({'id': test["id"], 'is_iceberg': test_predictions})
submission.to_csv('sub172.csv', index = False)
submission.head(10)

,id,is_iceberg
0,5941774d,0.026564
1,4023181e,0.371797
2,b20200e4,0.000140
3,e7f018bb,0.877413
4,4371c8c3,0.042351
5,a8d9b1fd,0.020246
6,29e7727e,0.079988
7,92a51ffb,0.969216
8,c769ac97,0.010322
9,aee0547d,0.000406


In [551]:
submission.shape

(8424, 2)

# New model with + featuremaps

In [72]:
acti = 'relu'

model = Sequential()

model.add(BatchNormalization(input_shape=(75, 75, 3)))

model.add(Conv2D(32, kernel_size=(3,3), activation=acti))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.3))


model.add(Conv2D(64, kernel_size=(3,3), activation=acti))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Conv2D(128, kernel_size=(3,3), activation=acti))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Conv2D(256, kernel_size=(3,3), activation=acti))
model.add(Conv2D(512, kernel_size=(3,3), activation=acti))
model.add(GlobalMaxPool2D())
model.add(BatchNormalization())
model.add(Dropout(0.3))


model.add(Dense(512, activation=acti))
model.add(Dropout(0.6)) 
model.add(BatchNormalization())


model.add(Dense(256, activation=acti))
model.add(Dropout(0.6))
model.add(BatchNormalization())


model.add(Dense(1, activation='sigmoid'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_411 (Bat (None, 75, 75, 3)         12        
_________________________________________________________________
conv2d_436 (Conv2D)          (None, 73, 73, 32)        896       
_________________________________________________________________
max_pooling2d_79 (MaxPooling (None, 36, 36, 32)        0         
_________________________________________________________________
batch_normalization_412 (Bat (None, 36, 36, 32)        128       
_________________________________________________________________
dropout_185 (Dropout)        (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_437 (Conv2D)          (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_80 (MaxPooling (None, 17, 17, 64)        0         
__________

In [85]:
train_gen = ImageDataGenerator(
        rotation_range=20,
        zoom_range = 0.1,
        width_shift_range=0.3,
        height_shift_range=0.3,
        rescale=1./255,
        fill_mode='nearest')


test_gen = ImageDataGenerator(rescale=1./255)


training_set = train_gen.flow(X_train, y_train, batch_size=264)
valid_set = test_gen.flow(X_valid, y_valid, batch_size=264)
test_set = test_gen.flow(X_test, batch_size=1, shuffle=False)

In [73]:
batch_size = 264
epochs = 80
steps_per_epoch=2**13/batch_size

model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.adam(lr=0.001, decay=0.005), metrics=['mae'])
model.fit_generator(training_set, steps_per_epoch=steps_per_epoch, epochs=epochs, 
                    callbacks=callbacks, validation_data=valid_set, validation_steps=400)

Epoch 1/80
31/31 [==============================] - 12s - loss: 0.7451 - mean_absolute_error: 0.4191 - val_loss: 0.6954 - val_mean_absolute_error: 0.4977
Epoch 2/80
31/31 [==============================] - 8s - loss: 0.6680 - mean_absolute_error: 0.3829 - val_loss: 0.7186 - val_mean_absolute_error: 0.5034
Epoch 3/80
31/31 [==============================] - 8s - loss: 0.5808 - mean_absolute_error: 0.3591 - val_loss: 0.7637 - val_mean_absolute_error: 0.5059
Epoch 4/80
31/31 [==============================] - 8s - loss: 0.5733 - mean_absolute_error: 0.3467 - val_loss: 1.2175 - val_mean_absolute_error: 0.5141
Epoch 5/80
31/31 [==============================] - 8s - loss: 0.5132 - mean_absolute_error: 0.3237 - val_loss: 1.0950 - val_mean_absolute_error: 0.5118
Epoch 6/80
 9/31 [=======>......................] - ETA: 1s - loss: 0.5527 - mean_absolute_error: 0.3179

KeyboardInterrupt: 

In [ ]:
print(model.evaluate_generator(training_set, steps_per_epoch))
print(model.evaluate_generator(valid_set, 400))

In [49]:
test_predictions = model.predict_generator(test_set, steps = test.shape[0])
test_predictions = test_predictions.reshape((test_predictions.shape[0]))
test_predictions = ['{:f}'.format(item) for item in test_predictions]

submission = pd.DataFrame({'id': test["id"], 'is_iceberg': test_predictions})
submission.to_csv('./submitions/sub098_32-64-128-256-512-1024.csv', index = False)
submission.head(10)

,id,is_iceberg
0,5941774d,0.000025
1,4023181e,0.998655
2,b20200e4,0.002371
3,e7f018bb,0.999963
4,4371c8c3,0.634913
5,a8d9b1fd,0.909739
6,29e7727e,0.000096
7,92a51ffb,0.999552
8,c769ac97,0.019017
9,aee0547d,0.000684


# Residual Network

In [87]:
from keras import layers
from keras import models


#
# image dimensions
#

img_height = 75
img_width = 75
img_channels = 3

#
# network params
#

cardinality = 1


def residual_network(x):
    """
    ResNeXt by default. For ResNet set `cardinality` = 1 above.
    
    """
    def add_common_layers(y):
        y = layers.BatchNormalization()(y)
        y = layers.LeakyReLU()(y)
        #y = layers.Dropout(0.4)(y)

        return y

    def grouped_convolution(y, nb_channels, _strides):
        # when `cardinality` == 1 this is just a standard convolution
        if cardinality == 1:
            return layers.Conv2D(nb_channels, kernel_size=(3, 3), strides=_strides, padding='same')(y)
        
        assert not nb_channels % cardinality
        _d = nb_channels // cardinality

        # in a grouped convolution layer, input and output channels are divided into `cardinality` groups,
        # and convolutions are separately performed within each group
        groups = []
        for j in range(cardinality):
            group = layers.Lambda(lambda z: z[:, :, :, j * _d:j * _d + _d])(y)
            groups.append(layers.Conv2D(_d, kernel_size=(3, 3), strides=_strides, padding='same')(group))
            
        # the grouped convolutional layer concatenates them as the outputs of the layer
        y = layers.concatenate(groups)

        return y

    def residual_block(y, nb_channels_in, nb_channels_out, _strides=(1, 1), _project_shortcut=False):
        """
        Our network consists of a stack of residual blocks. These blocks have the same topology,
        and are subject to two simple rules:
        - If producing spatial maps of the same size, the blocks share the same hyper-parameters (width and filter sizes).
        - Each time the spatial map is down-sampled by a factor of 2, the width of the blocks is multiplied by a factor of 2.
        """
        shortcut = y

        # we modify the residual building block as a bottleneck design to make the network more economical
        y = layers.Conv2D(nb_channels_in, kernel_size=(1, 1), strides=(1, 1), padding='same')(y)
        y = add_common_layers(y)

        # ResNeXt (identical to ResNet when `cardinality` == 1)
        y = grouped_convolution(y, nb_channels_in, _strides=_strides)
        y = add_common_layers(y)

        y = layers.Conv2D(nb_channels_out, kernel_size=(1, 1), strides=(1, 1), padding='same')(y)
        # batch normalization is employed after aggregating the transformations and before adding to the shortcut
        y = layers.BatchNormalization()(y)

        # identity shortcuts used directly when the input and output are of the same dimensions
        if _project_shortcut or _strides != (1, 1):
            # when the dimensions increase projection shortcut is used to match dimensions (done by 1×1 convolutions)
            # when the shortcuts go across feature maps of two sizes, they are performed with a stride of 2
            shortcut = layers.Conv2D(nb_channels_out, kernel_size=(1, 1), strides=_strides, padding='same')(shortcut)
            shortcut = layers.BatchNormalization()(shortcut)

        y = layers.add([shortcut, y])

        # relu is performed right after each batch normalization,
        # expect for the output of the block where relu is performed after the adding to the shortcut
        y = layers.LeakyReLU()(y)

        return y

    # conv1
    x = layers.Conv2D(32, kernel_size=(7, 7), strides=(2, 2))(x)
    x = add_common_layers(x)
    
    
    # conv2
    x = layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2))(x)
    for i in range(4):
        project_shortcut = True if i == 0 else False
        x = residual_block(x, 16, 32, _project_shortcut=project_shortcut)
        
    # conv2
    x = layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2))(x)
    for i in range(4):
        project_shortcut = True if i == 0 else False
        x = residual_block(x, 32, 64, _project_shortcut=project_shortcut)
    
    x = layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2))(x)
    x = layers.Conv2D(64, kernel_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = add_common_layers(x)
    


    #x = layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2))(x)
    #for i in range(2):
    #    project_shortcut = True if i == 0 else False
    #    x = residual_block(x, 64, 128, _project_shortcut=project_shortcut)


    # conv3
    #for i in range(4):
        # down-sampling is performed by conv3_1, conv4_1, and conv5_1 with a stride of 2
     #   strides = (2, 2) if i == 0 else (1, 1)
    #    x = residual_block(x, 256, 512, _strides=strides)

    # conv4
    #for i in range(6):
    #    strides = (2, 2) if i == 0 else (1, 1)
    #    x = residual_block(x, 512, 1024, _strides=strides)

    ## conv5
    #for i in range(3):
     #   strides = (2, 2) if i == 0 else (1, 1)
    #    x = residual_block(x, 1024, 2048, _strides=strides)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(1, activation='sigmoid')(x)
    return x


image_tensor = layers.Input(shape=(img_height, img_width, img_channels))
network_output = residual_network(image_tensor)
  
model = models.Model(inputs=[image_tensor], outputs=[network_output])
print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_31 (InputLayer)            (None, 75, 75, 3)     0                                            
____________________________________________________________________________________________________
conv2d_497 (Conv2D)              (None, 35, 35, 32)    4736        input_31[0][0]                   
____________________________________________________________________________________________________
batch_normalization_474 (BatchNo (None, 35, 35, 32)    128         conv2d_497[0][0]                 
____________________________________________________________________________________________________
leaky_re_lu_416 (LeakyReLU)      (None, 35, 35, 32)    0           batch_normalization_474[0][0]    
___________________________________________________________________________________________

In [88]:
batch_size = 264
epochs = 80
steps_per_epoch=2**16/batch_size #131072 samples

model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.adam(lr=0.01, decay=0.005), metrics=['mae'])
model.fit_generator(training_set, steps_per_epoch=steps_per_epoch, epochs=epochs, 
                    callbacks=callbacks, validation_data=valid_set, validation_steps=steps_per_epoch/10)

print(model.evaluate_generator(training_set, steps_per_epoch))
print(model.evaluate_generator(valid_set, steps_per_epoch/10))

Epoch 1/80
248/248 [==============================] - 50s - loss: 0.2708 - mean_absolute_error: 0.1698 - val_loss: 2.0550 - val_mean_absolute_error: 0.5244
Epoch 2/80
248/248 [==============================] - 44s - loss: 0.1483 - mean_absolute_error: 0.0903 - val_loss: 2.2274 - val_mean_absolute_error: 0.5357
Epoch 3/80
248/248 [==============================] - 72s - loss: 0.0959 - mean_absolute_error: 0.0575 - val_loss: 0.4138 - val_mean_absolute_error: 0.1869
Epoch 4/80
248/248 [==============================] - 44s - loss: 0.0638 - mean_absolute_error: 0.0380 - val_loss: 0.9665 - val_mean_absolute_error: 0.2194
Epoch 5/80
248/248 [==============================] - 45s - loss: 0.0419 - mean_absolute_error: 0.0248 - val_loss: 0.8404 - val_mean_absolute_error: 0.1762
Epoch 6/80
248/248 [==============================] - 45s - loss: 0.0354 - mean_absolute_error: 0.0204 - val_loss: 0.7721 - val_mean_absolute_error: 0.1425
Epoch 7/80
248/248 [==============================] - 45s - loss

In [70]:
batch_size = 64
epochs = 80
steps_per_epoch=2**16/batch_size #131072 samples

model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.adam(lr=0.01, decay=0.005), metrics=['mae'])
model.fit(X_train, y_train, epochs=epochs, batch_size = 32, 
                    callbacks=callbacks, validation_data=[X_valid, y_valid])

Train on 1203 samples, validate on 401 samples
Epoch 1/80
1203/1203 [==============================] - 6s - loss: 0.5884 - mean_absolute_error: 0.3730 - val_loss: 8.3605 - val_mean_absolute_error: 0.5187
Epoch 2/80
1203/1203 [==============================] - 1s - loss: 0.3766 - mean_absolute_error: 0.2469 - val_loss: 8.3605 - val_mean_absolute_error: 0.5187
Epoch 3/80
1203/1203 [==============================] - 1s - loss: 0.3273 - mean_absolute_error: 0.2052 - val_loss: 7.4735 - val_mean_absolute_error: 0.5071
Epoch 4/80
1203/1203 [==============================] - 2s - loss: 0.3077 - mean_absolute_error: 0.1835 - val_loss: 7.6541 - val_mean_absolute_error: 0.5183
Epoch 5/80
1203/1203 [==============================] - 1s - loss: 0.2680 - mean_absolute_error: 0.1742 - val_loss: 0.6835 - val_mean_absolute_error: 0.3062
Epoch 6/80
1203/1203 [==============================] - 1s - loss: 0.2227 - mean_absolute_error: 0.1384 - val_loss: 0.4018 - val_mean_absolute_error: 0.2089
Epoch 7/80


In [90]:
test_predictions = model.predict_generator(test_set, steps = test.shape[0])
test_predictions = test_predictions.reshape((test_predictions.shape[0]))
test_predictions = ['{:f}'.format(item) for item in test_predictions]

submission = pd.DataFrame({'id': test["id"], 'is_iceberg': test_predictions})
submission.to_csv('./submitions/sub132.csv', index = False)
submission.head(10)

,id,is_iceberg
0,5941774d,0.000000
1,4023181e,0.002718
2,b20200e4,0.000000
3,e7f018bb,0.990814
4,4371c8c3,0.000000
5,a8d9b1fd,0.000000
6,29e7727e,0.000007
7,92a51ffb,0.999999
8,c769ac97,0.000000
9,aee0547d,0.000006


In [131]:
train_gen = ImageDataGenerator(
        rotation_range=20,
        zoom_range = 0.1,
        width_shift_range=0.1,
        height_shift_range=0.1,
        vertical_flip=True,
        horizontal_flip=True,
        #rescale=1./255,
        fill_mode='nearest')


test_gen = ImageDataGenerator()


training_set = train_gen.flow(X_train, y_train, batch_size=32)


#model
model = Sequential()

model.add(Conv2D(16, kernel_size=(3,3), padding='same', input_shape=(75,75,2)))
model.add(BatchNormalization())
model.add(keras.layers.LeakyReLU())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, kernel_size=(5,5), padding='same'))
model.add(BatchNormalization())
model.add(keras.layers.LeakyReLU())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, kernel_size=(3,3), padding='same'))
model.add(BatchNormalization())
model.add(keras.layers.LeakyReLU())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, kernel_size=(3,3), padding='same'))
model.add(BatchNormalization())
model.add(keras.layers.LeakyReLU())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, kernel_size=(3,3), padding='same'))
model.add(BatchNormalization())
model.add(keras.layers.LeakyReLU())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, kernel_size=(3,3), padding='same'))
model.add(BatchNormalization())
model.add(keras.layers.LeakyReLU())
#model.add(MaxPooling2D(pool_size=(2,2)))


model.add(GlobalMaxPool2D())


model.add(Dense(1, activation='sigmoid'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_568 (Conv2D)          (None, 75, 75, 16)        304       
_________________________________________________________________
batch_normalization_546 (Bat (None, 75, 75, 16)        64        
_________________________________________________________________
leaky_re_lu_480 (LeakyReLU)  (None, 75, 75, 16)        0         
_________________________________________________________________
max_pooling2d_123 (MaxPoolin (None, 37, 37, 16)        0         
_________________________________________________________________
conv2d_569 (Conv2D)          (None, 37, 37, 32)        12832     
_________________________________________________________________
batch_normalization_547 (Bat (None, 37, 37, 32)        128       
_________________________________________________________________
leaky_re_lu_481 (LeakyReLU)  (None, 37, 37, 32)        0         
__________

In [125]:
batch_size = 32
epochs = 80
steps_per_epoch=2**14/batch_size 

callbacks = get_callbacks('./model.hdf5', patience=5)

model.compile(loss='mae', optimizer=keras.optimizers.adam(lr=0.01, decay=5e-5), metrics=['accuracy'])
model.fit_generator(training_set, steps_per_epoch=steps_per_epoch, epochs=epochs, 
                    callbacks=callbacks, validation_data=[X_valid, y_valid])

print(model.evaluate_generator(training_set, steps_per_epoch))
print(model.evaluate(X_testing, y_testing))

Epoch 1/80
512/512 [==============================] - 184s - loss: 0.3179 - acc: 0.6838 - val_loss: 0.3598 - val_acc: 0.6417
Epoch 2/80
512/512 [==============================] - 14s - loss: 0.2125 - acc: 0.7883 - val_loss: 0.2101 - val_acc: 0.7913
Epoch 3/80
512/512 [==============================] - 14s - loss: 0.1964 - acc: 0.8040 - val_loss: 0.2063 - val_acc: 0.7975
Epoch 4/80
512/512 [==============================] - 14s - loss: 0.1752 - acc: 0.8261 - val_loss: 0.2740 - val_acc: 0.7570
Epoch 5/80
512/512 [==============================] - 14s - loss: 0.1806 - acc: 0.8203 - val_loss: 0.3265 - val_acc: 0.6667
Epoch 6/80
512/512 [==============================] - 14s - loss: 0.1593 - acc: 0.8409 - val_loss: 0.1467 - val_acc: 0.8536
Epoch 7/80
512/512 [==============================] - 14s - loss: 0.1558 - acc: 0.8444 - val_loss: 0.1699 - val_acc: 0.8349
Epoch 8/80
512/512 [==============================] - 14s - loss: 0.1674 - acc: 0.8325 - val_loss: 0.1456 - val_acc: 0.8567
Epoch 9

In [132]:
batch_size = 32
epochs = 80
steps_per_epoch=2**16/batch_size #131072 samples

model.compile(loss='mae', optimizer=keras.optimizers.adam(lr=0.01, decay=5e-5), metrics=['mae'])
model.fit(X_train, y_train, epochs=epochs, batch_size = 32, 
                    callbacks=callbacks, validation_data=[X_valid, y_valid])

Train on 962 samples, validate on 321 samples
Epoch 1/80
962/962 [==============================] - 6s - loss: 0.3720 - mean_absolute_error: 0.3720 - val_loss: 0.5047 - val_mean_absolute_error: 0.5047
Epoch 2/80
962/962 [==============================] - 1s - loss: 0.2628 - mean_absolute_error: 0.2628 - val_loss: 0.5047 - val_mean_absolute_error: 0.5047
Epoch 3/80
962/962 [==============================] - 0s - loss: 0.1916 - mean_absolute_error: 0.1916 - val_loss: 0.4925 - val_mean_absolute_error: 0.4925
Epoch 4/80
962/962 [==============================] - 1s - loss: 0.2413 - mean_absolute_error: 0.2413 - val_loss: 0.4904 - val_mean_absolute_error: 0.4904
Epoch 5/80
962/962 [==============================] - 1s - loss: 0.1804 - mean_absolute_error: 0.1804 - val_loss: 0.4667 - val_mean_absolute_error: 0.4667
Epoch 6/80
962/962 [==============================] - 1s - loss: 0.1552 - mean_absolute_error: 0.1552 - val_loss: 0.5047 - val_mean_absolute_error: 0.5047
Epoch 7/80
962/962 [====

In [135]:
print(model.evaluate(X_train, y_train))
print(model.evaluate(X_testing, y_testing))

288/321 [=========================>....] - ETA: 0s[0.17678908915534569, 0.17678908915534569]


In [130]:
test_predictions = model.predict(X_test)
test_predictions = test_predictions.reshape((test_predictions.shape[0]))
test_predictions = ['{:f}'.format(item) for item in test_predictions]

submission = pd.DataFrame({'id': test["id"], 'is_iceberg': test_predictions})
submission.to_csv('./submitions/sub149.csv', index = False)
submission.head(10)

,id,is_iceberg
0,5941774d,0.000000
1,4023181e,0.999976
2,b20200e4,0.000000
3,e7f018bb,0.999727
4,4371c8c3,0.000000
5,a8d9b1fd,0.999999
6,29e7727e,0.000000
7,92a51ffb,1.000000
8,c769ac97,0.000000
9,aee0547d,0.000000


In [128]:
test_predictions.shape

(8424, 1)